In [ ]:
import yfinance as yf
import pandas as pd
# pip install yfinance
from pprint import pprint
from pandas_profiling import ProfileReport
import numpy as np
import scipy.stats as st
import plotly.graph_objects as go # for candle chart # next round future
import matplotlib.pyplot as plt
from datetime import datetime # convert timestamp and revese to string
%matplotlib inline
import quantstats as qs
import pandas_ta as ta
import math
import scipy.stats as st
%matplotlib notebook


# Down Load the SPY Data Frame

In [ ]:
# While closing price merely refers to the cost of shares at the end of the day, 
#the adjusted closing price considers other factors like dividends, stock splits,
#and new stock offerings. Since the adjusted closing price begins where the closing price ends,
#it can be called a more accurate measure of stocks' value.
SPYadj=yf.download("SPY")
SPYadj

# Time Handler function()1

In [ ]:
# from datetime import datetime, change time stap to str
def dftimeStaptoString(df):
    df.index.strftime("%Y-%m-%d")
    return df

# Time Handler function()2

In [ ]:
# cut the time frame/period for study needs
def timeHandlerFC(df):
    df=df.assign(year=df.index.year, month=df.index.month,day=df.index.day)
    return df

# DataFrame Time Slicer

In [ ]:
# study the index for and datetime libaray
def dataFrameYrselectionFC(df,time):
    df=df.loc[df.index>=time]
    return df

In [ ]:
SpyTrimmed2020=SPYadj.copy()
SpyTrimmed2020=dataFrameYrselectionFC(df=SpyTrimmed2020,time='2020-01-01')
SpyTrimmed2020
#perfect just like that, now we can manupulate any time we wanna with lots angels

# Build VolumeOverPriceTool

In [ ]:
# df:dataframe from yfinance
#x price interval for underlying price
#n slicer where interval start
#m slicer where interval end
# it build analysis base on close price and sum of the history vloume over price
# seek to find the signficance of price that interpreted by volume
def volumeonPriceintervalFC(df,x,n,m):
    binn=np.arange(0,480,x).tolist()
    result=pd.cut(df['Close'],bins=binn)
    df['Range']=result
    fm=df.groupby(['Range']).sum()['Volume'][n:m]  
    gp=fm.plot(kind='bar')
    return gp,fm
    
    

# Day counts up
   

In [ ]:
# we can consider add display to make it side by side
def consecutiveupdistributionFC(df):
    L=df['Adj Close'].tolist()
    dayupdistribution=[]
    adjcloseprior=L[0]
    count=0
    for x in range(len(L)-1):
        if L[x+1]>L[x]:
            count=count+1
        else:
            dayupdistribution.append(count)
            count=0
    dayupdistribution
    daycountdistributiondf=pd.DataFrame({"daycountoccurance":dayupdistribution})
    daycountdistributiondf["Occurance"]=0
    consecutiveupdstribution=daycountdistributiondf.groupby(['daycountoccurance']).count()
    consecutiveupdstribution
    nodownday=consecutiveupdstribution.copy().drop(0)
    nodownday['%ofwholeset']=round((nodownday['Occurance']/nodownday['Occurance'].sum())*100,2)
    gpp=nodownday['Occurance'].plot(kind='bar')
    gp=consecutiveupdstribution.plot(kind='bar')
    return gpp,nodownday,consecutiveupdstribution,gp


##need to add the format string
    

# Caculate rate of changes

In [ ]:
# notice a good way to understand how it work is to write a function.


# Caculate Return and Return distribution

In [ ]:
# have math dependency due to math.celi()
#Kurtosis -peak and tail of the return distribution; normal distribution have Kurtosis near 0
#imperical data may be more peaky with fater tail
#Skewness # negative side tail is fatter than postive side; extrem negative movement more likely
def Caculate_ReturnFC(df):
    df['return%']=((df['Close']-df['Open'])/df['Open']*100).round(2)
    dfraw=df['return%'].copy()
    x=df['return%'].max()
    y=df['return%'].min()
    describe=df['return%'].describe()
    skewness=st.skew(df['return%'].dropna())
    kurtosiss=st.kurtosis(df['return%'].dropna())
    # notcie shapiro not effective if n>5000
    #shapiro is a fair estimate on probability wise, still not a gurantee judgement
    # will sample 5000 a good way to go?
    p_value=st.shapiro(df['return%'].dropna().sample(5000))[1]
    # in scipy it return excess kurtosis as defualt
    print(f"daily return excess kurtosis(after -3 from normal) at {kurtosiss}")
    print(f"daily return skewness at {skewness}")
    print(f"daily return p_value at {p_value}")
    if p_value<=0.05:
        print("Null hypothesis of normality is rejected.")
    else:
        print("Null hypothesis of normality is accepted.")
    L=np.arange(y,math.ceil(x),1)
    df['daily_%return_range']=pd.cut(df['return%'],bins=L)
    gp=df.groupby(['daily_%return_range']).count()['Close'].plot(kind='bar')
    df['return%']=(df['return%']).astype(str)+'%'
    df=df.drop(columns='daily_%return_range')
    
    # {"stdv":[1,2,3],"normal:[]"}

    dfraww=dfraw.to_frame()
    stdd=d.std()
    meann=d.mean()
    #normal distribution prama
    y=[0.6820,0.9540,0.99]
    Distr=[]
    for x in y:
        m=x*len(SPYadj)
        Distr.append(m)
    Distr

    upper1std=meann+stdd
    lower1std=meann-stdd
    upper2std=meann+(2*stdd)
    lower2std=meann-(2*stdd)
    upper3std=meann+(3*stdd)
    lower3std=meann-(3*stdd)
    Distrbin=[f"{round(lower1std[0],4)} ~{round(upper1std[0],4)}",\
              f"{round(lower2std[0],4)}~{round(upper2std[0],4)}",\
              f"{round(lower3std[0],4)}~{round(upper3std[0],4)}"]
    a1=len(d.loc[(dfraww['return%']>lower2std[0] )& (dfraww['return%']<upper1std[0])])
    a2=len(d.loc[(dfraww['return%']>lower2std [0])& (dfraww['return%']<upper2std[0])])
    a3=len(d.loc[(dfraww['return%']>=lower2std[0] )& (dfraww['return%']<=upper3std[0])])
    actual=[]
    actual.append(a1)
    actual.append(a2)
    actual.append(a3)
    actual
    normalFreq=pd.DataFrame({'Distribin':Distrbin})
    normalFreq['Empirical']=actual
    normalFreq['Normal']=Distr
    normalFreq
    
    

    return df,gp,describe,normalFreq,dfraw
#we can unpack a4 for study 
# how can we create a normal distribution one to fit our data


In [ ]:
dfdraw=SPYadj['return%']
dfdraw.to_frame()

In [ ]:
# sknewness above 0 signal non-normality
#normal distribution tendency kurtosis at 3; while follow ticker indicate Leptokurtic, excess Kurtosis; think fat tail
a1,a2,a3,a4,a5=Caculate_ReturnFC(SPYadj)

In [ ]:
a4

In [ ]:
d

In [ ]:
len(SPYadj)

In [ ]:
(d>lower1std).count()

In [ ]:

d

In [ ]:
lower1std

In [ ]:

d.loc[(d['return%']>lower1std)]

In [ ]:
# {"stdv":[1,2,3],"normal:[]"}

# d.to_frame()
stdd=d.std()
meann=d.mean()
# 1std:{'1sigmaU':meann+stdd,'1sigmaL':meann-sttd}
y=[0.6820,0.9540,0.99]
Distr=[]

for x in y:
    m=x*len(SPYadj)
    Distr.append(m)
Distr

upper1std=meann+stdd
lower1std=meann-stdd
upper2std=meann+(2*stdd)
lower2std=meann-(2*stdd)
upper3std=meann+(3*stdd)
lower3std=meann-(3*stdd)
Distrbin=[f"{round(lower1std[0],4)} ~{round(upper1std[0],4)}",\
          f"{round(lower2std[0],4)}~{round(upper2std[0],4)}",\
          f"{round(lower3std[0],4)}~{round(upper3std[0],4)}"]
a1=len(d.loc[(d['return%']>lower2std[0] )& (d['return%']<upper1std[0])])
a2=len(d.loc[(d['return%']>lower2std [0])& (d['return%']<upper2std[0])])
a3=len(d.loc[(d['return%']>=lower2std[0] )& (d['return%']<=upper3std[0])])
actual=[]
actual.append(a1)
actual.append(a2)
actual.append(a3)
actual
normalFreq=pd.DataFrame({'Distribin':Distrbin})
normalFreq['Empirical']=actual
normalFreq['Normal']=Distr
normalFreq

In [ ]:
actual

In [ ]:
pd.DataFrame({1std:{'1sigmaU':meann+stdd,'1sigmaL':meann-sttd}})

# Calculate ATR- Historical vol

In [ ]:
# beautiful set up, it allow u to scope atr for realiize vol by anchor at today's date
def anchoredATRCalFC(df):
    df['TrueRange']=df['High']-df['Low']
    atrSpy=df.copy().ta.reverse.reset_index()
    rolling2D=[((atrSpy['TrueRange'][x]+atrSpy['TrueRange'][x+1])/2)/(atrSpy['Open'][x])for x in range(len(SPYadj)-1)]
    rolling2Draw=pd.Series(rolling2D)
    atrSpy['rolling2D']=(rolling2Draw*100).round(5).astype(str)+'%'
    
  #find out the atr change overtime from anchored date
    a={"years":["5days","20days","60days","1year","3year","5years","10years","20years","50years"]}
    b={"Days":[5,20,60,250,750,1250,2500,5000,12500]}
    c={"AvgDailyATRbaseonPeriod":[]}
    for x in b['Days']:
        #60period we need to add one digit; notice it start from 0 and exlude the last one on slicer,balance out
        periodmean=str(round((rolling2Draw[:x+1].mean()*100),3))+'%'
        c['AvgDailyATRbaseonPeriod'].append(periodmean)
    anchoredATR=pd.DataFrame({'years':a['years'],'Days':b['Days'],\
                              'AvgDailyATRbaseonPeriod':c['AvgDailyATRbaseonPeriod']}).T
    return atrSpy,rolling2Draw,anchoredATR

In [ ]:
aatr,batr,catr=anchoredATRCalFC(SPYadj)
aatr

In [ ]:
catr

# Caculate Log Return

# Build Graphic Generator

In [ ]:
# note input has to be us stock ticker a commpany wiht ""; eg input="SPY"
# start/end date has to be "YYYY-MM-DD" format
def givemeachart(input,start,end): # is this the dependency not within the function?# notice need to put diff quotation on it
    ticker=yf.Ticker(input)
    Fetched=ticker.history(start=start,end=end)
    plt.figure(figsize=(10,6))
    performance=plt.plot(Fetched['Close'])
    plt.grid()
    plt.show()
# looking for the documentary to define the functions

In [ ]:
SPYadj['return']=((SPYadj['Close']-SPYadj['Open'])/SPYadj['Open']*100).round(2).astype(str)+'%'
spyDaily=SPYadj.reset_index()
returndistribution=spyDaily.groupby('return').count().iloc[:,0]
returndistribution.plot(kind='bar')

# set_index(SPYadj['return']).value_counts()


In [ ]:
SPYadj.info()

In [ ]:
#None Null data, good quality 
SPYadj.dropna()

In [ ]:
# since it is binary event the distribution itself is selfexplantary; potential catched a meanreversal strategy 
# we can use value_counts(normialize=True) ---to give the % by apperance
a,b,c,d=consecutiveupdistributionFC(df=SPYadj)


In [ ]:
adjcloseprior=SPYadj['Adj Close'][0]
adjcloseprior

In [ ]:
(SPYadj['Adj Close'][1:]).tolist()

In [ ]:
# note start only configure the count start number of index, it always give out the full sets
L=SPYadj['Adj Close'].tolist()
dayupdistribution=[]
adjcloseprior=L[0]
count=0
for x in range(len(L)-1):
    if L[x+1]>L[x]:
        count=count+1
#         print(count)
    else:
        dayupdistribution.append(count)
        count=0
        
dayupdistribution
daycountdistributiondf=pd.DataFrame({"daycountoccurance":dayupdistribution})
daycountdistributiondf["helperoll"]=0
consecutiveupdstribution=daycountdistributiondf.groupby(['daycountoccurance']).count()
consecutiveupdstribution


In [ ]:
consecutiveupdstribution.plot(kind='bar')

In [ ]:
sd=SPYadj.copy().ta.strategy('ALL',timed=True)

In [ ]:
sd

# BollingerBand

In [ ]:
def showBbandsFC(df):
    a=df.ta.bbands(close='Adj Close',length=20,append=True)
    b=df[['BBU_20_2.0','Adj Close','BBL_20_2.0']]
    gp=b.plot()
    return gp,df

In [ ]:
showBbandsFC(df=SPYadj)

In [ ]:
f

In [ ]:
#https://www.vanguard.ca/en/advisor/products/products-group/etfs/VCN
CADadj=yf.download("VCN")
CADadj

In [ ]:
CADadj=yf.download("FTSE",period='40')
CADadj

# US vs Rest of the Word?

In [ ]:
#https://etfdb.com/index/msci-all-country-world-index-ex-usa/
restWord=yf.download("ACWX")
restWord

## add FRED API ? https://fred.stlouisfed.org/docs/api/fred/   \
## add TA-LIB for bollinger bands?


In [ ]:
df1=pd.DataFrame()
df1=df.ta.ticker('SPY',period='5y')  # note here we can even use ta module build on top on yfinace to have the ticker
df1
#but we would like to have the adj close so we use the download method

In [ ]:
df1=pd.DataFrame()
df1=df.ta.ticker('SPY',period='5y')  # note here we can even use ta module build on top on yfinace to have the ticker
df1

In [ ]:
df=pd.DataFrame()
df=pd.read_csv('SPYadj.csv')
df.ta.log_return(cumulative=True,append=True)
df # add the log return for the file

In [ ]:
strategy=df1.ta.categories
strategy


In [ ]:
help(ta.volatility)

In [ ]:
pov=SPYadj.ta.pvol().plot()
pov


In [ ]:
a=SPYadj.ta.bbands(close='Adj Close',length=20,append=True)
SPYadj

In [ ]:
a[[]]

In [ ]:
a[['BBU_20_2.0','BBL_20_2.0']].plot()

In [ ]:
help(qs.stats.conditional_value_at_risk) ## way u can use quanttats to learn concepts

In [ ]:
SPYadj.ta.tos_stdevall()

In [ ]:
result = ta.variancelatility(SPYadj.close)

In [ ]:
### need further study the way it is 
# potential option to overlay info ontop of it 
StudyProject=0
myStrategyBundle=ta.Strategy(
    name='StudyProject',
    ta=[
        {'kind':'donchian','lower_length':10,'upper_length':15},
        {'kind':'sma','lenght':10},
        
        
    ]

)
df.ta.strategy(StudyProject,length=15)

In [ ]:
help(ta.bbands)

In [ ]:
strategy[7]

In [ ]:
help(volatility)

In [ ]:
# have to reset the the index, note the document request to access date and price info
bollinger=SPYadj.ta.volatility.() # set 20 days for swing trading

In [ ]:
dfVol=df1.copy().ta.strategy("volatility")
dfVol
# need ta Dependency install to have it properly

In [ ]:
SPYadj.ta.strategy("All") # NEED TO INSTALL TA LIB

In [ ]:
help(ta.yf)

In [ ]:
help(df.ta)

In [ ]:
help(df.ta.indicators())

In [ ]:
help(ta.bbands)

In [ ]:
help(ta.squeeze) # here can tell more detail about it 

In [ ]:
qs.extend_pandas() # this is required to pass quantstat to Jupyter

In [ ]:
spyqt=qs.utils.download_returns('SPY')
spyqt

In [ ]:
#Calculate Sharp
qs.stats.sharpe(spyqt)

In [ ]:
spyqt.cagr()

In [ ]:
spyqt.plot_drawdown()

In [ ]:
spyqt.plot_distribution()

In [ ]:
spyqt.plot_drawdowns_periods()

In [ ]:
spyqt.monthly_returns()

In [ ]:
spyqt.plot_monthly_heatmap()

In [ ]:
spyqt.max_drawdown()

In [ ]:
#qs.reports.html(spyqt,output='fileloaction/spyreport.html')
qs.reports.plots(spyqt) # we can save as internet link format as well 

In [ ]:
spyqt.plot_earnings(start_balance=10000) # once qs pandas extend initiate all additional function rdy
# (savefig='filelocation') --can save the graph for ppt

In [ ]:
tickers={"SSO":0.3,"GOF":0.2,"BRW":0.2,"TLT":0.3}
profo=qs.utils.make_index(tickers)
qs.reports.html(profo,'SPY',output="profo.html")

In [ ]:
SPYadj.info()

In [ ]:
SPYadj.dropna()

In [ ]:
# note checked the time of data is time-stamp; hence it will be way easier to convert UTC format
SPYadj.index[0]

In [ ]:
SPYadj.assign(year=SPYadj.index.year, month=SPYadj.index.month,day=SPYadj.index.day)

In [ ]:
SPYadj.reset_index()

In [ ]:
split("1993-01-29",'-')

In [ ]:
fig=go.Figure(data=[go.Candlestick(x=SPYadj['Date'],open=SPYadj['Open'],high=SPYadj['High'],low=SPYadj['Low'],
                                  close=SPYadj['close'])
                 

In [ ]:
volumeonPriceintervalFC(df=SPYadj,x=1,n=380,m=400)

In [ ]:
var_spymaxperiod=np.var(SPYadj['Adj Close'])
std_spymaxperiod=np.std(SPYadj['Adj Close'])
mean_spymaxperiod=np.mean(SPYadj['Adj Close'])

In [ ]:
print(f"Roughly 68% among {mean_spymaxperiod-std_spymaxperiod*1} and {mean_spymaxperiod+std_spymaxperiod*1}")
print(f"Roughly 95% among {(mean_spymaxperiod-std_spymaxperiod*2)} and {mean_spymaxperiod+std_spymaxperiod*2}")
print(f"Roughly 99.7% among {mean_spymaxperiod-std_spymaxperiod*3} and {mean_spymaxperiod+std_spymaxperiod*3}")

In [ ]:
print(f"Roughly 68% among {mean_spymaxperiod-std_spymaxperiod*1} and {mean_spymaxperiod+std_spymaxperiod*1}")
print(f"Roughly 95% among 0 and {mean_spymaxperiod+std_spymaxperiod*2}")
print(f"Roughly 99.7% among 0 and {mean_spymaxperiod+std_spymaxperiod*3}")

In [ ]:
st.normaltest(SPYadj['Adj Close'].sample(1000))

In [ ]:
zSpyMaxperiod=st.zscore(SPYadj['Adj Close'])

In [ ]:
spy=yf.Ticker('SPY')
type(spy)

In [ ]:
spy.info

In [ ]:
spy.splits

In [ ]:
spy.info.keys() # show higher level frame, see what we can do about it

In [ ]:
# clean data
# deal with time serials of data

In [ ]:
spy.institutional_holders

In [ ]:
spy.earnings_history # need to fetch from bloomberg?

In [ ]:
FetchedSpy=spy.history(period='max')

In [ ]:
print(spy.history(period='max'))

In [ ]:
FetchedSpy.to_csv("SPYoutPut.csv")

In [ ]:
df=spy.dividends
df


In [ ]:
pprint(spy.info)

In [ ]:
IndexPE=spy.info['equityHoldings']['priceToEarnings']
IndexPE

In [ ]:
# for key,value in stockinfo.items():
#    print(key,':',value)
df=spy.dividends
dff=df.to_frame()
dff

In [ ]:
dir(df) # study these method for fun?

In [ ]:
data=df.resample('Y').sum()
data

In [ ]:
dff2=dff.reset_index()['Date'].dt.year # try how to split them further
dff2

In [ ]:
FetchedSpy

In [ ]:
FetchedSpy.shift(1)

In [ ]:
FetchedSpy.describe()

In [ ]:
FetchedSpy['Close'].max()

In [ ]:
volumefeedMaxperiod=FetchedSpy.groupby(['Close']).sum()['Volume']
volumefeedMaxperiod=volumefeedMaxperiod.reset_index()
volumefeedMaxperiod

In [ ]:
pricingbin=[0,25,50,75,100,125,150,175,200,225,250,275,300,325,350,375,400,425,450,475]
labels=["0~25","25~50","50~75","75~100","100~125","125~150","150~175","175~200","200~225","225~250",
      "250~275","275~300","300~325","325~350","350~375","375~400","400~425","425~450","450~int(FetchedSpy['Close'].max())"]
volumeOverpricelabels=pd.cut(volumefeedMaxperiod['Close'],bins=pricingbin)
volumeOverpricelabels

In [ ]:
volumefeedMaxperiod['StockRange']=volumeOverpricelabels
volumefeedMaxperiod

In [ ]:
pricevolumeforPlot=volumefeedMaxperiod.groupby(['StockRange']).sum()['Volume']
pricevolumeforPlot.plot(kind='bar')

In [ ]:
FetchedSpy['Volume'].plot() 

In [ ]:
closerlookvolume=pricevolumeforPlot.loc[pricevolumeforPlot.index[10:]]
closerlookvolume.plot(kind='bar')


In [ ]:
byunit10=np.linspace(start=0,stop=480,num=49)
byunit10

In [ ]:
FetchedSpy

In [ ]:
np.arange(0,480,1)

In [ ]:
FetchedSpy

In [ ]:
FetchedSpy

In [ ]:
def volumeonPriceintervalFC(df,x,n,m):
    binn=np.arange(0,480,x).tolist()
    result=pd.cut(df['Close'],bins=binn)
    df['Range']=result
    fm=df.groupby(['Range']).sum()['Volume'][n:m]  
    gp=fm.plot(kind='bar')
    return gp
    

In [ ]:
volumeonPriceintervalFC(df=FetchedSpy,x=1,n=450,m=480)

In [ ]:
type(FetchedSpy['Volume'][0])

In [ ]:
binn=np.arange(25,480,3).tolist()
binn
result=pd.cut(FetchedSpy['Close'],bins=binn)
result
FetchedSpy['Range']=result
plott=FetchedSpy.groupby(['Range']).sum()['Volume'][130:]
plott.plot(kind='bar')

In [ ]:
labelsBY5=[]

In [ ]:
volumeOverpricelabelsbyUnit10=pd.cut(volumefeedMaxperiod['Close'],bins=byunit10)
volumeOverpricelabelsbyUnit10

In [ ]:
volumeOverpricelabelsbyUnit10df=volumefeedMaxperiod.copy()
volumeOverpricelabelsbyUnit10df['StockRange']=volumeOverpricelabelsbyUnit10
volumeOverpricelabelsbyUnit10df

In [ ]:
voulumeoverPriceUnit10=volumeOverpricelabelsbyUnit10df.groupby(["StockRange"]).sum()['Volume']
voulumeoverPriceUnit10.plot(kind='bar')

In [ ]:
len(voulumeoverPriceUnit10)

In [ ]:
closerlookunitby10graph=voulumeoverPriceUnit10[30:].plot(kind="bar")

In [ ]:

volumeOverpricelabelsbyUnit10=pd.cut(volumefeedMaxperiod['Close'],bins=byunit10)

In [ ]:
#looking closer


In [ ]:
df2022=spy.history(start='2022-01-01',end='2022-12-30', rounding=True) # note here rounding =true to give to cents
df2022

In [ ]:
profileMaxPeriod=ProfileReport(FetchedSpy,title="MaxPeriodReport")
profileMaxPeriod

In [ ]:
import matplotlib.pyplot as plt

In [ ]:

plt.figure(figsize=(10,6))
performance=plt.plot(FetchedSpy['Close'])
plt.grid()
plt.show()

In [ ]:
def givemeachart(input,start,end): # is this the dependency not within the function?# notice need to put diff quotation on it
    ticker=yf.Ticker(input)
    Fetched=ticker.history(start=start,end=end)
    plt.figure(figsize=(10,6))
    performance=plt.plot(Fetched['Close'])
    plt.grid()
    plt.show()
# looking for the documentary to define the functions

In [ ]:
graph2022=givemeachart("de",'2022-01-01','2022-12-30')

In [ ]:
graphTqqq=givemeachart("TQQQ",'2022-01-01','2022-12-30')

In [ ]:
Bitcoin=givemeachart('BTC-USD','2022-01-01','2022-12-30')

In [ ]:
#faster way to do things
# from pandas_datareader import data as pdr

# import yfinance as yf
# yf.pdr_override() # <== that's all it takes :-)

# # download dataframe
# data = pdr.get_data_yahoo("SPY", start="2017-01-01", end="2017-04-30")

In [ ]:
# note we can request the data from yahoo finace by per minutes
qqq=yf.Ticker('QQQ')
df=qqq.history(period='max',actions=False,rounding=True,interval='15m')
df

In [ ]:
df15min=qqq.history(period='60d',actions=False,rounding=True,interval='15m',prepost=False)
df15min

In [ ]:
df15min=qqq.history(period='60d',actions=False,rounding=True,interval='15m',prepost=False,start="2022-12-01",end='2022-01-01')
df15min

In [ ]:
df15min=qqq.history(period='60d',actions=False,rounding=True,interval='15m',prepost=False)
df15min

In [ ]:
df15min.info()

In [ ]:
# note it give the index loop through time

In [ ]:
#discover it is a 
type(df15min.copy().reset_index()['Datetime'][0])

In [ ]:
4
# stack overflow

# First, make sure your TIMESTAMP column is a datetime instead of a string (e.g. df['TIMESTAMP'] = pd.to_datetime(df.TIMESTAMP)).

# Next, use this column as the index of the dataframe. To make this permanent, df.set_index('TIMESTAMP, inplace=True)`.

# Now you can resample for any given frequency (e.g. 30min) and use different methods of aggregation such as sum, mean (the default), a lambda function, etc).

# Optionally, you can add .fillna(0) to replace the NaNs with zeros.

In [ ]:
ploTSeries=df15min['Close']
ploTSeries

In [ ]:
plt.figure(figsize=(8,5))
ploTSeries.plot(title='QQQ15min')

In [ ]:
YMDformatSPYadj

In [ ]:
howmanydayfromtoday=(np.arange(0,len(YMDformatSPYadj))+1).tolist()
howmanydayfromtoday=howmanydayfromtoday[::-1]
howmanydayfromtoday# no we can construct it for the day we wanna

In [ ]:
YMDformatSPYadj['Retriveday']=howmanydayfromtoday
Yeartodayframe=YMDformatSPYadj.sort_values(by='Retriveday',ascending=True)
Yeartodayframe

In [ ]:
resetdf=YMDformatSPYadj.reset_index()
resetdf

In [ ]:
spy=yf.Ticker('SPY')
type(spy)

In [ ]:
time in df.reset_index()['Date'] if 

In [ ]:
finalize15min=[]
for i,day in enumerate(Yeartodayframe.reset_index()['Date'],start=1):
#          print(i,day)
    if (i%60==0 & i>=59):
        start=resetdf.iloc[i-59,0].strftime("%Y-%m-%d")
        end=resetdf.iloc[i,0].strftime("%Y-%m-%d")
        df=spy.history(start=start,end=end,actions=False,rounding=True,interval='15m',prepost=False)
        #?loop through df append each raw
        #? append each columns to list and let overwrite?
        #any better way to do ?D

        


        
        


# df15mins=spy.history(start='2022-10-30',end='2022-11-30',actions=False,rounding=True,interval='15m',prepost=False)
# df15mins

In [ ]:
df15mins=qqq.history(period="90d",actions=False,rounding=True,interval='15m',prepost=False)

In [ ]:
current_date=datetime.now()
current_date

In [ ]:
df15mins.index.strftime("%Y-%m-%d")[0]

In [ ]:
# lower level data for deeper loop, can be start data=x x from a range listed

# Option Section

In [ ]:
Experisionlist=spy.options
Experisionlist

In [ ]:
Experisionlist[0]

In [ ]:
puts=options.puts
puts

In [ ]:
options.calls.info()

In [ ]:
options.puts.info()

In [ ]:

for index,price in enumerate(c):
    if ((price%10==0) & (price>=10)):
        labelsBY5.append(f"less than {price}")
labelsBY5
    
        
           


            
        